In [2]:
import cv2
from keras.models import load_model
import numpy as np
import os

D:\ProgramData\Miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model = load_model('emojinator.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()


0
1
2
3
4
5
6
7
8
9
10
11
12
4 0.508083
7 0.9998882
7 0.9791443
7 0.9791443
7 0.6997637
7 0.6997637
10 0.99885666
10 0.99885666
7 0.99630034
7 0.99630034
7 0.99921906
7 0.99921906
7 0.9948408
7 0.9948408
7 0.99922156
7 0.99922156
7 0.99840885
7 0.99840885
7 0.9994357
7 0.9994357
7 0.94466996
7 0.94466996
7 0.999374
7 0.999374
7 0.9771862
7 0.9771862
7 0.9975738
7 0.9975738
7 0.990325
7 0.990325
7 0.99991786
7 0.99991786
7 0.9998807
7 0.9998807
7 0.9993461
7 0.9993461
7 0.9998983
7 0.9998983
7 0.99985313
7 0.99985313
7 0.9994382
7 0.9994382
3 0.50282556
3 0.50282556
10 0.31231996
10 0.31231996
7 0.99937695
7 0.99937695
7 0.99996936
7 0.99996936
2 0.52937037
2 0.52937037
7 0.9955395
7 0.9955395
2 0.7831071
2 0.7831071
7 0.9999701
7 0.9999701
7 0.9999347
7 0.9999347
2 0.68108195
2 0.68108195
2 0.39593378
2 0.39593378
7 0.9999131
7 0.9999131
7 0.8059476
7 0.8059476
7 0.9999937
7 0.9999937
3 0.33755904
3 0.33755904
7 0.9999665
7 0.9999665
7 0.99344665
7 0.99344665
7 0.9999658
7 0.9999658
7 

9 0.91453034
9 0.91453034
9 0.6161821
9 0.6161821
6 0.3139568
6 0.3139568
9 0.9045951
9 0.9045951
9 0.92616665
9 0.92616665
3 0.8714771
3 0.8714771
9 0.44996566
9 0.44996566
11 0.7362064
11 0.7362064
1 0.7809005
1 0.7809005
11 0.62009853
11 0.62009853
11 0.8954289
11 0.8954289
1 0.7292092
1 0.7292092
1 0.93297035
1 0.93297035
1 0.70456886
1 0.70456886
1 0.5927751
1 0.5927751
1 0.9951687
1 0.9951687
1 0.9197899
1 0.9197899
9 0.9706633
9 0.9706633
4 0.79611295
4 0.79611295
6 0.98269147
6 0.98269147
9 0.55504686
9 0.55504686
7 0.8264651
7 0.8264651
4 0.888923
4 0.888923
6 0.34071848
6 0.34071848
6 0.9418915
6 0.9418915
9 0.48234016
9 0.48234016
9 0.99903774
9 0.99903774
9 0.8167158
9 0.8167158
6 0.84153116
6 0.84153116
9 0.95450866
9 0.95450866
11 0.939664
11 0.939664
11 0.6860042
11 0.6860042
3 0.5140293
3 0.5140293
11 0.37665018
11 0.37665018
3 0.34660923
3 0.34660923
6 0.6765126
6 0.6765126
2 0.23439081
2 0.23439081
9 0.6270468
9 0.6270468
9 0.53252023
9 0.53252023
9 0.87191755
9 0.871

9 0.9993048
9 0.9993048
9 1.0
9 1.0
9 0.5832089
9 0.5832089
6 0.7625739
6 0.7625739
6 0.9987979
6 0.9987979
6 0.97761744
6 0.97761744
9 0.9999999
9 0.9999999
6 0.9973628
6 0.9973628
6 0.9990613
6 0.9990613
6 0.9993691
6 0.9993691
6 0.9083121
6 0.9083121
6 0.9958943
6 0.9958943
9 0.5556219
9 0.5556219
9 0.9090267
9 0.9090267
6 0.93075955
6 0.93075955
9 0.5039834
9 0.5039834
6 0.58308893
6 0.58308893
6 0.6504442
6 0.6504442
6 0.6628976
6 0.6628976
6 0.8672922
6 0.8672922
6 0.7021405
6 0.7021405
6 0.64684147
6 0.64684147
6 0.962379
6 0.962379
6 0.95882046
6 0.95882046
9 0.6117071
9 0.6117071
6 0.99924964
6 0.99924964
9 0.9976375
9 0.9976375
9 0.5994664
9 0.5994664
6 0.99991095
6 0.99991095
9 0.98637867
9 0.98637867
4 0.95332587
4 0.95332587
2 0.8769307
2 0.8769307
7 0.9300851
7 0.9300851
4 0.9979759
4 0.9979759
10 0.49506903
10 0.49506903
3 0.5119644
3 0.5119644
3 0.7986386
3 0.7986386
2 0.91283345
2 0.91283345
3 0.6377234
3 0.6377234
2 0.457651
2 0.457651
4 0.56062174
4 0.56062174
3 0.81

6 0.6599324
6 0.6599324
9 0.9999981
9 0.9999981
4 0.7857176
4 0.7857176
4 0.7937814
4 0.7937814
4 0.8884244
4 0.8884244
4 0.47965047
4 0.47965047
2 0.974246
2 0.974246
2 0.7106758
2 0.7106758
10 0.35972
10 0.35972
3 0.49829033
3 0.49829033
2 0.5819869
2 0.5819869
6 0.9942221
6 0.9942221
6 0.96172404
6 0.96172404
3 0.5465725
3 0.5465725
3 0.50064135
3 0.50064135
9 0.9999957
9 0.9999957
3 0.5708752
3 0.5708752
3 0.5938815
3 0.5938815
6 0.36111373
6 0.36111373
6 0.9795903
6 0.9795903
6 0.66395986
6 0.66395986
10 0.4597479
10 0.4597479
4 0.6222323
4 0.6222323
6 0.93843526
6 0.93843526
2 0.91859496
2 0.91859496
4 0.94783276
4 0.94783276
4 0.43841037
4 0.43841037
6 0.7783913
6 0.7783913
6 0.83676386
6 0.83676386
2 0.74614257
2 0.74614257
10 0.4507617
10 0.4507617
4 0.9971558
4 0.9971558
4 0.5954308
4 0.5954308
10 0.3758948
10 0.3758948
10 0.55779374
10 0.55779374
6 0.99622715
6 0.99622715
6 0.79290694
6 0.79290694
6 0.9150871
6 0.9150871
6 0.99259186
6 0.99259186
6 0.682163
6 0.682163
6 0.88

2 0.41948426
2 0.41948426
10 0.7452392
10 0.7452392
3 0.47180584
3 0.47180584
4 0.6984067
4 0.6984067
10 0.38577443
10 0.38577443
3 0.47899938
3 0.47899938
10 0.35453442
10 0.35453442
8 0.7860959
8 0.7860959
2 0.9870228
2 0.9870228
3 0.36749035
3 0.36749035
10 0.6598559
10 0.6598559
4 0.7333471
4 0.7333471
10 0.47340778
10 0.47340778
2 0.5541938
2 0.5541938
2 0.6238504
2 0.6238504
2 0.71195716
2 0.71195716
4 0.8455572
4 0.8455572
3 0.71037304
3 0.71037304
10 0.6032868
10 0.6032868
2 0.411131
2 0.411131
2 0.38424128
2 0.38424128
8 0.5505802
8 0.5505802
10 0.66521245
10 0.66521245
4 0.5257459
4 0.5257459
8 0.6720477
8 0.6720477
2 0.31955108
2 0.31955108
2 0.5468323
2 0.5468323
10 0.43222678
10 0.43222678
10 0.47086582
10 0.47086582
8 0.83049726
8 0.83049726
2 0.8108353
2 0.8108353
10 0.56507194
10 0.56507194
10 0.48186165
10 0.48186165
9 0.9997099
9 0.9997099
4 0.9993086
4 0.9993086
2 0.61738974
2 0.61738974
4 0.99915695
4 0.99915695
4 0.99998033
4 0.99998033
4 0.4621402
4 0.4621402
4 0.

...

5 0.4439951
5 0.4439951
5 0.6242939
5 0.6242939
5 0.98690224
5 0.98690224
5 0.9958688
5 0.9958688
5 0.9779961
5 0.9779961
2 0.62216777
2 0.62216777
5 0.8934981
5 0.8934981
5 0.96331716
5 0.96331716
2 0.59559846
2 0.59559846
5 0.6556023
5 0.6556023
2 0.9850129
2 0.9850129
5 0.6408595
5 0.6408595
2 0.71988124
2 0.71988124
2 0.66880995
2 0.66880995
2 0.98797786
2 0.98797786
2 0.984669
2 0.984669
2 0.98993486
2 0.98993486
2 0.9779804
2 0.9779804
2 0.7966419
2 0.7966419
2 0.86570233
2 0.86570233
5 0.5253699
5 0.5253699
2 0.66962725
2 0.66962725
2 0.7111433
2 0.7111433
2 0.59218913
2 0.59218913
2 0.65399563
2 0.65399563
2 0.77139384
2 0.77139384
2 0.8112425
2 0.8112425
2 0.6575461
2 0.6575461
2 0.9119698
2 0.9119698
5 0.626557
5 0.626557
2 0.9814347
2 0.9814347
5 0.7358074
5 0.7358074
2 0.4697113
2 0.4697113
2 0.659181
2 0.659181
2 0.97503567
2 0.97503567
2 0.9202687
2 0.9202687
2 0.92916095
2 0.92916095
2 0.7856617
2 0.7856617
2 0.9864668
2 0.9864668
2 0.8800466
2 0.8800466
2 0.7552115
2 0.

5 0.99953556
5 0.99953556
5 0.99773335
5 0.99773335
5 0.9956655
5 0.9956655
9 0.9999981
9 0.9999981
5 0.99934393
5 0.99934393
5 0.9986657
5 0.9986657
5 0.9637645
5 0.9637645
5 0.99997973
5 0.99997973
5 0.99969554
5 0.99969554
5 0.99996114
5 0.99996114
5 0.85288054
5 0.85288054
5 0.99934715
5 0.99934715
5 0.83233535
5 0.83233535
5 0.98233676
5 0.98233676
5 0.99596965
5 0.99596965
5 0.99987674
5 0.99987674
9 0.99554235
9 0.99554235
5 0.9921483
5 0.9921483
5 0.9897955
5 0.9897955
5 0.9922954
5 0.9922954
5 0.5447681
5 0.5447681
5 0.99929345
5 0.99929345
5 0.9997696
5 0.9997696
5 0.9999056
5 0.9999056
5 0.9998178
5 0.9998178
5 0.99994206
5 0.99994206
9 0.9450489
9 0.9450489
5 0.9652425
5 0.9652425
5 0.7935413
5 0.7935413
5 0.88219863
5 0.88219863
5 0.9999591
5 0.9999591
5 0.8130055
5 0.8130055
5 0.9994646
5 0.9994646
5 0.9856764
5 0.9856764
5 0.89520675
5 0.89520675
5 0.99990475
5 0.99990475
5 0.999739
5 0.999739
5 0.9726548
5 0.9726548
5 0.8868113
5 0.8868113
9 0.9967795
9 0.9967795
9 0.99

5 0.98236644
5 0.98236644
5 0.9995173
5 0.9995173
6 0.5265474
6 0.5265474
5 0.9999391
5 0.9999391
5 0.96606654
5 0.96606654
5 0.9989429
5 0.9989429
5 0.9993368
5 0.9993368
5 0.98734796
5 0.98734796
6 0.9648801
6 0.9648801
6 0.866747
6 0.866747
5 0.99160427
5 0.99160427
5 0.99919194
5 0.99919194
5 0.9972766
5 0.9972766
5 0.99939036
5 0.99939036
5 0.99895275
5 0.99895275
5 0.9994491
5 0.9994491
5 0.99230725
5 0.99230725
5 0.99756706
5 0.99756706
5 0.9992036
5 0.9992036
5 0.9999919
5 0.9999919
5 0.99270016
5 0.99270016
6 0.7159208
6 0.7159208
5 0.96521354
5 0.96521354
5 0.9999354
5 0.9999354
5 0.9856542
5 0.9856542
5 0.9999306
5 0.9999306
5 0.99988127
5 0.99988127
5 0.99956506
5 0.99956506
5 0.9996475
5 0.9996475
5 0.99511445
5 0.99511445
5 0.99973637
5 0.99973637
6 0.7958642
6 0.7958642
5 0.99966884
5 0.99966884
5 0.99941504
5 0.99941504
5 0.8937378
5 0.8937378
5 0.6310213
5 0.6310213
5 0.9846096
5 0.9846096
5 0.99950325
5 0.99950325
5 0.99989605
5 0.99989605
6 0.7069465
6 0.7069465
5 0.

5 0.9981862
5 0.9981862
5 0.97996736
5 0.97996736
5 0.9998099
5 0.9998099
5 0.8676372
5 0.8676372
6 0.650304
6 0.650304
5 0.9994548
5 0.9994548
6 0.54753083
6 0.54753083
6 0.85795397
6 0.85795397
5 0.99969006
5 0.99969006
5 0.99922705
5 0.99922705
5 0.9996679
5 0.9996679
5 0.9997999
5 0.9997999
5 0.9979563
5 0.9979563
5 0.9997669
5 0.9997669
5 0.921564
5 0.921564
5 0.99882907
5 0.99882907
5 0.99973935
5 0.99973935
9 0.7516203
9 0.7516203
5 0.999046
5 0.999046
9 0.982515
9 0.982515
5 0.73232704
5 0.73232704
5 0.99995756
5 0.99995756
6 0.5427352
6 0.5427352
5 0.9940183
5 0.9940183
5 0.99716836
5 0.99716836
5 0.9988518
5 0.9988518
5 0.9998185
5 0.9998185
5 0.8040274
5 0.8040274
5 0.9992254
5 0.9992254
5 0.9999707
5 0.9999707
5 0.9998462
5 0.9998462
5 0.99964833
5 0.99964833
5 0.9998534
5 0.9998534
5 0.98934376
5 0.98934376
9 0.90234256
9 0.90234256
5 0.9805997
5 0.9805997
6 0.9512175
6 0.9512175
5 0.9995772
5 0.9995772
5 0.99517846
5 0.99517846
5 0.8414433
5 0.8414433
5 0.9841261
5 0.9841

KeyboardInterrupt: 